<a href="https://colab.research.google.com/github/mot1122/study_pytorch/blob/main/2_7_SSD_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
cp -r /content/drive/MyDrive/pytorch_advanced/2_objectdetection/utils/ utils

In [7]:
import os.path as osp
import random
import time

import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.utils.data as data

In [8]:
torch.manual_seed(1234)
np.random.seed(1234)
random.seed(1234)

In [9]:
device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"device: {device}")

device: cpu


In [10]:
from utils.ssd_model import make_datapath_list,VOCDataset,DataTransform,Anno_xml2list,od_collate_fn
rootpath="/content/drive/MyDrive/pytorch_advanced/2_objectdetection/data/VOCdevkit/VOC2012/"
train_img_path,train_anno_path,val_img_path,val_anno_path=make_datapath_list(rootpath)
voc_classes= ['aeroplane', 'bicycle', 'bird', 'boat',
               'bottle', 'bus', 'car', 'cat', 'chair',
               'cow', 'diningtable', 'dog', 'horse',
               'motorbike', 'person', 'pottedplant',
               'sheep', 'sofa', 'train', 'tvmonitor']
color_mean = (104, 117, 123)  # (BGR)の色の平均値
input_size = 300  # 画像のinputサイズを300×300にする
train_dataset=VOCDataset(train_img_path,train_anno_path,phase="train",transform=DataTransform(input_size,color_mean),transform_anno=Anno_xml2list(voc_classes))
val_dataset=VOCDataset(val_img_path,val_anno_path,phase="val",transform=DataTransform(input_size,color_mean),transform_anno=Anno_xml2list(voc_classes))
batch_size=32
train_dataloader=data.DataLoader(train_dataset,batch_size=batch_size,shuffle=True,collate_fn=od_collate_fn)
val_dataloader=data.DataLoader(val_dataset,batch_size=batch_size,shuffle=False,collate_fn=od_collate_fn)
dataloader_dict={"train":train_dataloader,"val":val_dataloader}

In [13]:
from utils.ssd_model import SSD
ssd_cfg={
    'num_classes': 21,  # 背景クラスを含めた合計クラス数
    'input_size': 300,  # 画像の入力サイズ
    'bbox_aspect_num': [4, 6, 6, 6, 4, 4],  # 出力するDBoxのアスペクト比の種類
    'feature_maps': [38, 19, 10, 5, 3, 1],  # 各sourceの画像サイズ
    'steps': [8, 16, 32, 64, 100, 300],  # DBOXの大きさを決める
    'min_sizes': [30, 60, 111, 162, 213, 264],  # DBOXの大きさを決める
    'max_sizes': [60, 111, 162, 213, 264, 315],  # DBOXの大きさを決める
    'aspect_ratios': [[2], [2, 3], [2, 3], [2, 3], [2], [2]],
}
net=SSD(phase="train",cfg=ssd_cfg)
vgg_weights=torch.load("/content/drive/MyDrive/pytorch_advanced/2_objectdetection/weights/vgg16_reducedfc.pth")
net.vgg.load_state_dict(vgg_weights)

def weights_init(m):
  if isinstance(m,nn.Conv2d): 
    init.kaiming_normal_(m.weight.data)
    if m.bias is not None: nn.init.constant_(m.bias,0.0)

net.extras.apply(weights_init)
net.loc.apply(weights_init)
net.conf.apply(weights_init)

device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")
print(f"device: {device}")
print("loaded weghts")

device: cuda:0
loaded weghts


In [15]:
from utils.ssd_model import MultiBoxLoss

criterion=MultiBoxLoss(jaccard_thresh=0.5,neg_pos=3,device=device)
optimizer=optim.SGD(net.parameters(),lr=1e-3,momentum=0.9,weight_decay=5e-4)

In [3]:
def train_model(net,dataloaders_dict,criterion,optimizer,num_epochs):
  device=torch.device("cuda:0" if torch.cuda.is_available else "cpu")
  print(f"device: {device}")
  net.to(device)
  iterattion=1
  epoch_train_loss=0.0
  epoch_val_loss=0.0
  logs=[]
  for epoch in range(num_epochs+1):
    t_epoch_start=time.time()
    t_iter_start=time.time()
    print(f"{epoch}/{num_epochs}")
    for phase in ["train","val"]:
      if phase=="train":
        net.train()
        print("train")
      else:
        if ((epoch+1)%10==0):
          net.eval()
          print("val")
        else:continue
      for images,targets in dataloaders_dict["phase"]:
        images=images.to(device)
        targets=[anno.to(device) for anno in targets]
        optimizer.zero_grad()
        with torch.set_grad_enabled(phase="train"):
          outputs=net(images)
          loss_l,loss_c=criterion(outputs,targets)
          loss=loss_l+loss_c
          if phase=="train":
            loss.backward()
            nn.utils.clip_grad_value_(net.parameters(),clip_value=2.0)
            optimizer.step()
            if iteration%10==0:
              t_iter_finish=time.time()
              print(f"iteration: {iteration}, loss: {loss}, {t_iter_finish-t_iter_start}")
              t_iter_start=time.time()
            epoch_train_loss+=loss.item()
            iteration+=1
          else:epoch_val_loss+=loss.item()
    t_epoch_finish=time.time()
    print(f"epoch: {epoch}, train loss: {epoch_train_loss}, val loss: {epoch_val_loss}, {t_epoch_finish-t_epoch_start}sec")
    t_epoch_start=time.time()
    log_epoch={"epoch":epoch+1, "train_loss":epoch_train_loss, "val_loss":epoch_val_loss}
    logs.append(log_epoch)
    df=pd.DataFrame(logs)
    df.to_csv("log_output.csv")
    epoch_train_loss,epoch_val_loss=0.0,0.0
    if ((epoch+1)%10==0):torch.save(net.state_dict(),'weights/ssd300_' + str(epoch+1) + '.pth')

In [ ]:
num_epochs=50
train_model(net, dataloaders_dict, criterion, optimizer, num_epochs=num_epochs)